In [5]:
import os
import sys
import shutil
from traceback import print_exc
from distutils.core import setup
from distutils.extension import Extension

from Cython.Build import cythonize
from Cython.Distutils import build_ext
# 获取相对路径



In [6]:
def pytoso(name):
    try:
        if name.endswith('.py'):
            sys.argv = ['main.py', "build_ext", '--inplace']
            module_name = name.split('.')[0]
            ext_modules = [
                Extension(module_name, [name], extra_compile_args=['-O3']),
            ]
            relative_dir, _ = os.path.split(name)
            setup(
                name='MyProgram',
                cmdclass={'build_ext': build_ext},
                ext_modules=cythonize(ext_modules, language_level=3),
                package_dir={'': os.path.join('src', relative_dir)},
            )
    except Exception as e:
        print_exc()
        return 1
    return 0

In [ ]:
if __name__ == '__main__':

    names = []
    for root, dirs, files in os.walk('..'):
        for name in files:
            if name.endswith('.py'):
                names.append(os.path.join(root, name))

    for name in names:
        try:
            status = pytoso(name)
        except Exception as e:
            print_exc()
        else:
            if status:
                print(f'Failed to convert {name}')
            else:
                print(f'Converted {name}')
        finally:
            try:
                os.remove(f'script_code/{name.replace(".py", ".c")}')
            except:
                pass
    try:
        shutil.rmtree('build')
    except:
        pass

    names = []
    for root, dirs, files in os.walk('.'):
        for name in files:
            if not name.endswith('__init__.py'):
                names.append(os.path.join(root, name))
    for name in names:
        if name.endswith('.pyx') or name.endswith('__init__.py'):
            relative_dir, filename = os.path.split(name)
            if name.endswith('.pyx'):
                filename = filename.replace('.pyx', '.so')
            initial_path = os.path.join(relative_dir, filename)
            try:
                shutil.move(initial_path, os.path.join('src', initial_path))
            except Exception:
                pass
        elif name.endswith('.c') or name.endswith('.cpp') or name.endswith('.html'):
            try:
                os.remove(name)
            except:
                pass